版权所有 (c) MONAI Consortium  
根据 Apache 许可证 2.0 版（以下简称“许可证”）授权许可；  
除非遵守许可证，否则你不能使用此文件。  
你可以在以下网址获得许可证副本：  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
除非适用法律要求或书面同意，依据许可证分发的软件按“原样”分发，  
不附带任何明示或暗示的担保或条件。  
请参阅许可证以了解具体语言及权限限制。

# 2D 模型在 3D 体积上的推理

用例：一个 2D 模型，例如 2D 分割 U-Net，在 2D 输入上操作，这些输入可以是来自 3D 体积（例如 CT 扫描）的切片。

在按照本教程编辑滑动窗口推理器后，它可以处理整个流程，如下所示：

![image](https://github.com/Project-MONAI/tutorials/blob/main/figures/2d_inference_3d_input.png?raw=1)

输入是一个 *3D 体积*，一个 *2D 模型*，输出是一个带有 2D 切片预测聚合的 *3D 体积*。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/tutorials/blob/main/modules/2d_inference_3d_volume.ipynb)


## 设置环境

In [2]:
!python -c "import monai" || pip install -q "monai-weekly[tqdm]"

2024-12-25 01:20:58.440718: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-25 01:20:58.463292: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-25 01:20:58.470474: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-25 01:20:58.487863: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-25 01:20:59.848112: W tensorflow/compiler/tf2

## 设置导入


In [3]:
import torch

from monai.config import print_config
from monai.inferers import SliceInferer
from monai.networks.nets import UNet

print_config()

MONAI version: 1.5.dev2451
Numpy version: 1.26.4
Pytorch version: 2.5.1+cu121
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 7c1c58cd10db72c01b5cdda1600cd68e262437cf
MONAI __file__: /usr/local/lib/python3.10/dist-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.3.2
scikit-image version: 0.25.0
scipy version: 1.13.1
Pillow version: 11.0.0
Tensorboard version: 2.17.1
gdown version: 5.2.0
TorchVision version: 0.20.1+cu121
tqdm version: 4.67.1
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.5
pandas version: 2.2.2
einops version: 0.8.0
transformers version: 4.47.1
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.
clearml version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.i

## SliceInferer
实现此功能的最简单方法是扩展 `monai.inferers` 中的 `SlidingWindowInferer`。在 MONAI 中，这个功能作为 `SliceInferer` 提供（[详细信息](https://docs.monai.io/en/latest/inferers.html#sliceinferer)）。

## 用法

In [4]:
# 创建一个用于测试目的的随机初始化权重的 2D UNet 3 层网络，每层通过因子 2 进行下采样/上采样，并具有 2 卷积残差单元
net = UNet(
    spatial_dims=2,
    in_channels=1,
    out_channels=1,
    channels=(4, 8, 16),
    strides=(2, 2),
    num_res_units=2,
)

# 初始化一个形状为 (N,C,D,H,W) 的虚拟 3D 张量体积 (N,C,D,H,W)
input_volume = torch.ones(1, 1, 64, 256, 256)

# 创建一个 SliceInferer 实例，设置 roi_size 为 256x256（HxW），并在 D 轴上滑动。
axial_inferer = SliceInferer(roi_size=(256, 256), sw_batch_size=1, cval=-1, progress=True)

output = axial_inferer(input_volume, net)

# 输出是一个带有 2D 切片聚合的 3D 体积。
print("Axial Inferer Output Shape: ", output.shape)
# 创建一个 `SliceInferer` 实例，设置 `roi_size` 为 64x256（DxW），并在 H 轴上滑动。
coronal_inferer = SliceInferer(
    roi_size=(64, 256),
    sw_batch_size=1,
    spatial_dim=1,  # 在这里添加沿切片的空间维度。
    cval=-1,
    progress=True,
)

output = coronal_inferer(input_volume, net)

# 输出是一个带有 2D 切片聚合的 3D 体积。
print("Coronal Inferer Output Shape: ", output.shape)

100%|██████████| 64/64 [00:01<00:00, 33.29it/s]


Axial Inferer Output Shape:  torch.Size([1, 1, 64, 256, 256])


100%|██████████| 256/256 [00:01<00:00, 129.78it/s]

Coronal Inferer Output Shape:  torch.Size([1, 1, 64, 256, 256])


Note that with `axial_inferer` and `coronal_inferer`, the number of inference iterations is 64 and 256 respectively.